In [ ]:
 # -*- coding: utf-8 -*-
#
# Author: Milan Ondrasovic <milan.ondrasovic@gmail.com>
#
# MIT License
#
# Copyright (c) 2021 Milan Ondrašovič
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

import json
import pickle
import pathlib
import numpy as np
import dataclasses
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('tableau-colorblind10')
pd.set_option('display.float_format', lambda x: '%.5f' % x)
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 11

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
def read_otb_report(performance_file_path):
    with open(performance_file_path, 'rt') as in_file:
        performance = json.load(in_file)

    top_level_key = next(iter(performance.keys()))
    overall_performance = performance[top_level_key]['overall']

    return overall_performance

def extract_context_from_name(tracker_name):
    tokens = tracker_name.split('_')
    batch_size = int(tokens[-2])
    epoch = int(tokens[-1])
    return batch_size, epoch

def read_otb_reports(reports_dir_path):
    data = []

    for tracker_report_dir in pathlib.Path(reports_dir_path).iterdir():
        performance_file_path = str(tracker_report_dir / "performance.json")
        overall_performance = read_otb_report(performance_file_path)

        batch_size, epoch = extract_context_from_name(tracker_report_dir.stem)

        record = {'batch_size': batch_size, 'epoch': epoch}
        stats = (
            'success_score', 'precision_score', 'success_rate', 'speed_fps')
        record.update({item: overall_performance[item] for item in stats})
        data.append(record)

    return pd.DataFrame(data)

reports_dir_path = "../reports/OTB2013"
stats = read_otb_reports(reports_dir_path)
stats